Wingate et al., AIstat_summary 2011, Algorithm 4
---

This paper by Wingate et al. proposes methods for recording, editing, and playing-back program traces, which become core components in PTO.

Algorithm 4 in this paper is "An illustration how the number and type of random choices can change." It is a short program which implicitly defines a distribution. 

The algorithm is (in Matlab):

```
1: m=poissrnd();
2: for i=1:m
3:     X(i) = gammarnd();
4: end;
5: for i=m+1:2*m
6:     X(i) = randn;
7: end;
```

It's intended as a distribution we might want to sample from, but for PTO purposes, let's define a fitness function on it -- `sum` -- and try to optimise it. The reason it's interesting for both Wingate et al. and for us is that the number `m` can change, which, in a naive implementation of program traces, would mean we were interpreting old trace material, previously used to sample from a gamma distribution, to sample from a normal, which would lead to bad results. Wingate et al. propose a better program trace implementation which avoids that, and which we therefore adopt in PTO.


Implementing the problem for PTO
---

We first define the problem by defining the generator and fitness. Numpy provides the functions we need, but we are not using Numpy in our PTO generator code, for now (we will still use it for analysis, though). Python's standard library `random` module provides `gammavariate` and `normalvariate`, but it doesn't provide a Poisson variate, so we will implement that first.

In [1]:
import random, math
def poissonvariate(lamb):
    """A method which will be slow for large lambda
    From https://www.johndcook.com/blog/2010/06/14/generating-poisson-random-values/
init:
         Let L ← exp(−λ), k ← 0 and p ← 1.
do:
         k ← k + 1.
         Generate uniform random number u in [0,1] and let p ← p × u.
while p > L.
return k − 1.

>>> sum([poissonvariate(2.5) for _ in range(100)]) / 100.0
 -> a value near 2.5
    """
    L = math.exp(-lamb)
    k = 0
    p = 1
    while p > L:
        k += 1
        u = random.random()
        p *= u
    return k - 1

Then, the generator closely mimics the original Matlab. Fitness is just `sum` as stated.

In [2]:
def generator():
    m = poissonvariate(10)
    X = []
    for i in range(m):
        X.append(random.gammavariate(1, 1)) # alpha=1, beta=1
    for i in range(m):
        X.append(random.normalvariate(0, 1))
    return X
fitness = sum

In [3]:
from PTO import random, random_function, solve

In [4]:
ind, fit = solve(generator, fitness, solver="HC", effort=1.0)
print(ind)
print("fitness: ", fit)

[1.1250017242418833, 0.03440916362225633, 1.2527401514967758, 0.48426153719285503, 1.3708654354886314, 1.4606277159874475, 0.06843229168290292, 1.705244945114607, 2.436036978341712, 0.7599906487459528, 0.10948734179255812, 1.0363381356236474, 0.899048506757045, 0.1323435110086408, 1.895726098059353, 1.234083009302191, 0.35502190034742154, 0.9826755349121823, 1.44459866916048, 0.4628223350521674, 0.2253260210722508, 1.1179371324600886, 0.6812351732496652, 0.5325693343224871, 0.731307848540265, -0.3667340228207865, 1.587646711796417, 0.4199073112597792]
('fitness: ', 24.17895114381088)


Experiments and Analysis
---

Next, we can do a large run and carry out some analysis on our results. 
We'll import functions provided by PTO, and also use scipy.


In [5]:
from PTO import compare_all, stat_summary
import scipy.stats # only for post-run analysis

In [6]:
# compare_all: you can pass in sizes if needed, else None. It uses 'budget', not 'effort'.
results = compare_all(fitness, [generator], sizes=None, 
                      methods=None, budget=10000, num_runs=10)

(None, False, 'generator', 'RS') 38.0116251102
(None, False, 'generator', 'HC') 131.285579646
(None, False, 'generator', 'LA') 172.664080657
(None, False, 'generator', 'EA') 129.231888812
(None, False, 'generator', 'PS') 118.8642199
(None, True, 'generator', 'RS') 43.3675204453
(None, True, 'generator', 'HC') 367.445714938
(None, True, 'generator', 'LA') 274.095546235
(None, True, 'generator', 'EA') 265.053609808
(None, True, 'generator', 'PS') 192.573180364
(None, False, 'generator', 'RS') 37.4300091569
(None, False, 'generator', 'HC') 143.025177524
(None, False, 'generator', 'LA') 136.14143454
(None, False, 'generator', 'EA') 136.556102761
(None, False, 'generator', 'PS') 156.080370431
(None, True, 'generator', 'RS') 39.8088577916
(None, True, 'generator', 'HC') 362.931336414
(None, True, 'generator', 'LA') 317.081544872
(None, True, 'generator', 'EA') 307.182717566
(None, True, 'generator', 'PS') 212.334785172
(None, False, 'generator', 'RS') 38.0253001762
(None, False, 'generator',

**Experiment 1**: Compare structured trace with linear, using hill-climbing: structured trace wins by far, of course.

In [9]:
d0, d1 = results[(None, False, 'generator', 'HC')], results[(None, True, 'generator', 'HC')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean 140.04 std 18.02 min 110.51 med 137.16 max 169.31
mean 339.96 std 17.52 min 306.86 med 341.46 max 367.45
Ttest_indResult(statistic=-23.866841778779722, pvalue=4.4656991378244708e-15)


**Experiment 2**: Compare hill-climbing with EA, using structured trace: HC wins!

In [10]:
d0, d1 = results[(None, True, 'generator', 'HC')], results[(None, True, 'generator', 'EA')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean 339.96 std 17.52 min 306.86 med 341.46 max 367.45
mean 278.86 std 16.90 min 259.91 med 274.59 max 307.18
Ttest_indResult(statistic=7.5306762107943719, pvalue=5.7340017105491142e-07)


**Experiment 3**: Compare HC with LAHC, using structured trace: HC wins! Stupid algorithms are good with the right representation?!

In [11]:
d0, d1 = results[(None, True, 'generator', 'HC')], results[(None, True, 'generator', 'LA')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean 339.96 std 17.52 min 306.86 med 341.46 max 367.45
mean 295.84 std 23.81 min 260.69 med 291.51 max 343.54
Ttest_indResult(statistic=4.4785064029570272, pvalue=0.00029045136637852127)


Conclusions and future work
---

We have demonstrated the use of variable-length and heterogeneously-typed (ie different distributions) program traces, and how to carry out an experiment using PTO's provided tools, leading to some simple experiment results.

Some ideas to test in future:

* Try an alternative generator which interleaves the gamma and normal sampling.

* Instead of setting `m` at the start, decide "whether to stop" at each
iteration -- need to make sure `m` distribution is still Poisson, though.
